# **Aplicaciones de la Teoría de Dualidad**

**Ejercicio 1 (Precio en un mercado marginalista):** Considera el problema de funcionamiento de un mercado eléctrico con el que hemos trabajado en la práctica anterior, donde los costes de producción de los $N_G$ productores que conforman el mercado vienen dados mediante funciones convexas lineales por tramos, en particular,

<center>

|  Productor    | Tramo 1   | Tramo 2  | Tramo 3 |
| ------------  | --------     | ---      | ---    |  
| 1            | (50, 10)      | (20, 15) | (30, 45)|
| 2            | (10, 5)       | (50, 26) | (60, 55)|   
| 3            | (100, 25)     | (50, 27) | (50, 30)|   

</center>

Recuerda que la tabla muestra los pares $(l_{i,j}, m_{i,j})$, $\forall i = 1, \ldots, N_{G}$, $\forall j = 1, \ldots, N_{p_{i}}$ que representan las longitudes y pendientes de los trozos lineales que determinan tales funciones de coste.

1.   Calcula la variable dual asociada (llamémosle $\lambda$) a la ecuación de equilibrio (o de cierre de mercado) para un valor de demanda $D = 200.$




In [2]:
%pip install -q amplpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 33.2 MB/s eta 0:00:00


In [3]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

Mercado = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


Construimos el modelo.

In [4]:
Mercado.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set Prod;                       # Conjunto de productores
set Tram;                       # Conjunto de tramos de las funciones de coste
param D >= 0;                   # Demanda del mercado
param lon {Prod, Tram} >= 0;    # Longitud de cada tramo
param slope {Prod, Tram} >= 0;  # Pendiente de cada tramo

var p {i in Prod, j in Tram} >= 0, <= lon[i,j];  # Potencia producida por cada
                                                 # productor en cada tramo

minimize coste_total:
    sum{i in Prod, j in Tram} slope[i,j]*p[i,j];
s.t. balance: sum{i in Prod, j in Tram} p[i,j] = D;
""")

Introducimos los datos necesarios para resolver el modelo de cierre de mercado.

In [5]:
def preparar_datos_mercado():
    import pandas as pd
    import numpy as np

    # Conjunto de tramos o piezas:
    tramos = ["T1", "T2", "T3"]

    # Conjunto de productores:
    productores = ["G1", "G2", "G3"]

    # Demanda del mercado:
    demanda = 200

    # Creamos un data frame que contenga la longitud de los tramos
    lon_df = pd.DataFrame(
        np.array(
            [
                [50, 20, 30],
                [10, 50, 60],
                [100, 50, 50],
            ]
        ),
        columns= tramos,
        index  = productores,
    )

    # Creamos un data frame que contenga las pendientes de los tramos
    slope_df = pd.DataFrame(
        np.array(
            [
                [10, 15, 45],
                [5, 26, 55],
                [25, 27, 30],
            ]
        ),
        columns= tramos,
        index  = productores,
    )

    return demanda, tramos, productores, lon_df, slope_df


In [6]:
# Cargamos los datos:

# Load the data from pandas.DataFrame objects:
demanda, tramos, productores, lon_df, slope_df = preparar_datos_mercado()

Mercado.set["Prod"] = productores
Mercado.set["Tram"] = tramos
Mercado.get_parameter("lon").set_values(lon_df)
Mercado.get_parameter("slope").set_values(slope_df)
Mercado.get_parameter("D").set(demanda)


Resolvemos:

In [7]:
Mercado.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert Mercado.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1:  - Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
 - CPXPARAM_Simplex_Display                         0
 - CPXPARAM_MIP_Display                             0
 - CPXPARAM_Barrier_Display                         0
CPLEX 22.1.1: optimal solution; objective 3870
0 simplex iterations


In [8]:
p = Mercado.get_variable("p") # Producción por generador y tramo
df_p = p.get_values().to_pandas()
coste_total = Mercado.get_objective("coste_total").value()
lam = Mercado.get_constraint("balance").dual()
df_p = df_p.round(3)
print(df_p)
print(f"Coste total: {coste_total:.3f}")
print(f"Lambda: {lam:.1f}")

# Resumimos lo que produce cada productor:
df_Pg = df_p.groupby(level='index0').sum()
print(df_Pg)

               p.val
index0 index1       
G1     T1         50
       T2         20
       T3          0
G2     T1         10
       T2         20
       T3          0
G3     T1        100
       T2          0
       T3          0
Coste total: 3870.000
Lambda: 26.0
        p.val
index0       
G1         70
G2         30
G3        100


Calcula el coste total de funcionamiento del mercado para $D=200$, $D=201$ y $D=199$.

In [9]:
Tcost = {}
for demanda in [200, 201, 199]:
    Mercado.get_parameter("D").set(demanda)
    Mercado.solve(solver="cplex")
    Tcost[f"{demanda}"] = Mercado.get_objective("coste_total").value()


CPLEX 22.1.1:  - Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
 - CPXPARAM_Simplex_Display                         0
 - CPXPARAM_MIP_Display                             0
 - CPXPARAM_Barrier_Display                         0
CPLEX 22.1.1: optimal solution; objective 3870
0 simplex iterations
CPLEX 22.1.1:  - Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
 - CPXPARAM_Simplex_Display                         0
 - CPXPARAM_MIP_Display                             0
 - CPXPARAM_Barrier_Display                         0
CPLEX 22.1.1: optimal solution; objective 3896
0 simplex iterations
CPLEX 22.1.1:  - Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
 - CPXPARAM_Simplex_Display                         0
 - CPXPARAM_MIP_Display                             0
 - CPXPARAM_Barrier_Display                         0
CPLEX 22.1.1: optimal solution; objective 3844
0 simplex iterations


In [10]:
print(Tcost)

{'200': 3870.0, '201': 3896.0, '199': 3844.0}


¿Qué puedes decir acerca de cómo varía dicho coste? ¿Con qué está relacionada dicha variación?

La variación en el coste total de producción es precisamente la variable dual $\lambda$.

**Ejercicio 2 (Teoría de juegos - Equilibrio de mercado):** Considera el problema que habría de resolver cada uno de los $N_G$ productores de electricidad que participan en el mercado para decidir la cantidad óptima de electricidad que deberían vender en dicho mercado si el precio de liquidación es $\lambda$.


1.   Formula dicho problema como un problema de Programación Matemática.

Considera el producto $i$ con $i \leq N_G$. Dicho productor (que consideramos *racional*) persigue maximizar su beneficio (esto es, los ingresos por la venta de electricidad menos los costes de producirla). Naturalmente, el productor no puede producir más allá de su capacidad. Consecuentemente, la toma de decisiones del productor $i$ se puede modelar mediante el siguiente LP:

$$
\begin{array}{lll}
    &\max           & \lambda \, P_i - \sum_{j=1}^{N_{p_{i}}} m_{ij}p_{ij}      &    \\
    &\textrm{s.t.}  & P_{i} =  \sum_{j=1}^{N_{p_{i}}} p_{ij}   &    \\
    &               & 0 \leq p_{ij} \geq l_{ij}, \quad \forall j = 1,\ldots, N_{p_{i}} &    
\end{array}
$$

2. Resuelve el problema anterior utilizando *amplpy* para cada uno de los tres productores que constituyen el mercado.

In [11]:
Productor = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [12]:
Productor.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set Tram;                       # Conjunto de tramos de las funciones de coste
param lambda;                   # Variable dual lambda
param lon {Tram} >= 0;          # Longitud de cada tramo
param slope {Tram};             # Pendiente de cada tramo

var p {j in Tram} >= 0, <= lon[j];  # Potencia producida por cada
                                    # productor en cada tramo

var P >= 0;

maximize beneficio: lambda * P - sum{j in Tram} slope[j]*p[j];
s.t. prod_total: sum{j in Tram} p[j] = P;
""")

Cargamos los datos para los parámetros de entrada del modelo (objeto de AMPL) "Productor". Nótese que estos parámetros de entrada son, en esencia, un subconjunto de los utilizados por el modelo "Mercado", en particular, aquellos que hacen referencia a un productor específico $i$, con $i \leq N_{G}$.

Por tanto, a continuación, resolveremos el modelo "Productor" para todo $i \leq N_{G}$.

In [13]:
import pandas as pd

P = {} # En este diccionario almacenaremos la cantidad total producida por cada
       # productor

p = pd.DataFrame(columns = tramos) # En este dataframe alacenaremos lo que
                                   # produce cada productor de cada uno de lo
                                   # tramos de su función de costes convexa
                                   # lineal a trozos

for i in productores:
  # Cargamos los datos referidos al productor i:
  Productor.set["Tram"] = tramos
  Productor.get_parameter("lon").set_values(lon_df.loc[i])
  Productor.get_parameter("slope").set_values(slope_df.loc[i])
  Productor.get_parameter("lambda").set(lam) # La variable dual asociada a la ecuación de equilibrio de "Mercado"

  # Resolvemos el modelo "Productor" para el productor i:
  Productor.solve(solver="highs") # Resolvemos con el solver "cplex"
  assert Productor.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente
  P[f"{i}"] = Productor.get_variable("P").value()
  p.loc[f"{i}"] = Productor.get_variable("p").get_values().to_pandas()["p.val"]
print(lam)
print(P)
print(p)

HiGHS 1.7.1: HiGHS 1.7.1: optimal solution; objective 1020
0 simplex iterations
0 barrier iterations
HiGHS 1.7.1: HiGHS 1.7.1: optimal solution; objective 210
0 simplex iterations
0 barrier iterations
HiGHS 1.7.1: HiGHS 1.7.1: optimal solution; objective 100
0 simplex iterations
0 barrier iterations
26.0
{'G1': 70.0, 'G2': 60.0, 'G3': 100.0}
     T1  T2  T3
G1   50  20   0
G2   10  50   0
G3  100   0   0


*¿Qué puedes decir acerca de las cantidades óptimas obtenidas?*

Obsérvese que el problema LP que modela al productor G2 tiene múltiples soluciones óptimas. Veámoslo. Para ello, vamos a determinar, de entre todas las soluciones óptimas disponibles (todas ellas con valor objetivo de 210), aquella que hace el menor uso del tramo T2 de producción. De hecho, esta solución, junto con la obtenida en la celda anterior, conforman los puntos extremos del segmento de soluciones óptimas que posee el problema del productor G2.

In [14]:
Productor_G2 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [15]:
Productor_G2.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set Tram;                       # Conjunto de tramos de las funciones de coste
param lambda;                   # Variable dual lambda
param beneficio_min;            # Beneficio mínimo que imponemos a la solución
param lon {Tram} >= 0;          # Longitud de cada tramo
param slope {Tram};             # Pendiente de cada tramo

var p {j in Tram} >= 0, <= lon[j];  # Potencia producida por cada
                                    # productor en cada tramo

var P >= 0;

minimize Produccion: P;          # Buscamos la SO que minimiza la producción
                                 # del productor G2
s.t.  b_min:  lambda * P - sum{j in Tram} slope[j]*p[j] >= beneficio_min; # Beneficio mínimo que imponemos a la solución
      prod_total: sum{j in Tram} p[j] = P;
""")

In [16]:
# Cargamos los datos referidos al productor G2:
Productor_G2.set["Tram"] = tramos
Productor_G2.get_parameter("lon").set_values(lon_df.loc["G2"])
Productor_G2.get_parameter("slope").set_values(slope_df.loc["G2"])
Productor_G2.get_parameter("lambda").set(lam) # La variable dual asociada a la ecuación de equilibrio de "Mercado"
Productor_G2.get_parameter("beneficio_min").set(210) # Imponemos el beneficio de 210 para garantizar optimalidad

# Resolvemos el modelo "Productor_G2":
Productor_G2.solve(solver="highs") # Resolvemos con el solver "highs"
assert Productor_G2.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente
print(Productor_G2.get_variable("P").value())
print(Productor_G2.get_variable("p").get_values().to_pandas()["p.val"])


HiGHS 1.7.1: HiGHS 1.7.1: optimal solution; objective 10
0 simplex iterations
0 barrier iterations
10.0
T1    10
T2     0
T3     0
Name: p.val, dtype: int64


Por tanto, al productor G2 le es *indiferente* producir en el rango que va de 10 a 60, por lo que si el precio de liquidación del mercado se fija a 26 (variable dual de la ecuación de equilibrio de mercado), las soluciones óptimas de los tres productores coincidirán con la dictada por el mercado. Dicho de otro modo, la variable dual de la ecuación de balance del mercado constituye un precio de equilibrio para el cual los tres productores y el mercado (que tiene como misión satisfacer la demanda de 200) se sienten satisfechos (pues todos ellos satisfacen sus objetivos o intereses particulares).

**Ejercicio 3 (Optimización Robusta):** Considera el problema de la dieta con el que trabajamos en la Práctica 1 (en su variante más sencilla). Supón que, por una concatenación de fallos técnicos, no se ha podido medir correctamente en el laboratorio la cantidad de fósforo que contiene cada uno de los tipos de alimentos disponibles para conformar la dieta. Por tanto, la tabla nutrientes-alimentos presenta "missing data" (o datos faltantes), tal y como sigue, donde "NaN" significa "Not a Number".

<center>

|  Nutriente    | Cantidad <br />  requerida   | Maíz A   | Avena  | Maíz B  | Salvado | Linaza|
| ------------  | --------     | ---      | ---    | ---     | ----    | ---   |
| DN            | 74.2         | 78.6     | 70.1   | 80.1    | 67.2    | 77.0  |
| DP            | 14.7         | 6.50     | 9.40   | 8.80    | 13.7    | 30.4  |
| Ca            | 0.14         | 0.02     | 0.09   | 0.03    | 0.14    | 0.41  |
| Ph            | 0.55         | NaN     | NaN   | NaN    | NaN    | NaN  |

</center>

No obstante, un estudio estadístico efectuado sobre análisis de estos alimentos realizados en el pasado revela que:


1.   La *suma* de una unidad de cada uno de los cinco alimentos disponibles contiene al menos dos unidades de fósforo.
2.   El salvado siempre contiene más del doble de fósforo que la avena y el maíz B juntos.
3.   La linaza siempre contiene más de 1.5 veces fósforo que el maíz A y el maíz B juntos.
4.   Una unidad de maíz A más una unidad de linaza contienen al menos una unidad de fósforo.

Se pide:


1.   Formular un problema de Programación Matemática que permita determinar las cantidades de alimentos necesarias para *garantizar* los aportes mínimos de nutrientes exigidos en la dieta con el *menor coste* posible.





Llamemos $a_{4j}$ a la cantidad de fósforo que aporta el alimento $j \in \{mA, A, mB, S, L\}$. Queremos que
$$a_{4mA}x_{mA} + a_{4A}x_{A} + a_{4mB}x_{mB} +a_{4S}x_{S} + a_{4L}x_{L} \geq 0.55$$

teniendo en cuenta que

$$
\begin{array}{rcl}
    a_{4mA} + a_{4A} + a_{4mB} + a_{4L} + a_{4S} & \geq      & 2   \\
    a_{4S}  & \geq   & 2(a_{4A} + a_{4mB})    \\
    a_{4L}  & \geq   & 1.5(a_{mA} + a_{4mB})    \\
    a_{4mA} + a_{4L} & \geq      & 1  
\end{array}
$$

El *peor caso* de aporte de fósforo que se puede dar con la combinación de alimentos se corresponde con el mínimo de

$$ a_{4mA}x_{mA} + a_{4A}x_{A} + a_{4mB}x_{mB} +a_{4S}x_{S} + a_{4L}x_{L}$$

Por tanto, lo que buscamos es que el siguiente problema LP en las variables $a_{4j}$, $j \in \{mA, A, mB, S, L\}$

$$
\begin{array}{rrcl}
    \min & a_{4mA}x_{mA} + a_{4A}x_{A} + a_{4mB}x_{mB} +a_{4S}x_{S} + a_{4L}x_{L}\\
    {\rm sujeto \ a} & a_{4mA} + a_{4A} + a_{4mB} + a_{4S} + a_{4L} & \geq      & 2   \\
    &a_{4S} -2(a_{4A} + a_{4mB}) & \geq   & 0    \\
    &a_{4L}  - 1.5(a_{mA} + a_{4mB})& \geq   & 0    \\
    &a_{4mA} + a_{4L} & \geq      & 1  \\
    &a_{4mA}, a_{4A}, a_{4mB}, a_{4S}, a_{4L} & \geq & 0
\end{array}
$$
sea $\geq 0.55$.

O mejore dicho, buscamos las cantidades $x_{mA}, x_{A}, x_{mB}, x_{S}, x_{L}$ de alimentos que hemos de aportar a la dieta para que dicho mínimo sea $\geq 0.55$ a la vez que minimizamos el coste de la misma.

Podemos calcular el dual del problema anterior, que tiene la siguiente pinta

$$
\begin{array}{rrcl}
    \max & 2y_1 + y_4\\
    {\rm sujeto \ a} & y_1-1.5y_3+y_4 & \leq      & x_{mA}   \\
    &y_1-2y_2 & \leq   & x_{A}    \\
    &y_1-2y_2-1.5y_3& \leq   & x_{mB}    \\
    &y_1+y_2 & \leq      & x_{S}  \\
    &y_1+y_3 + y_{4} & \leq      & x_{L}  \\
    &y_1, y_2, y_3, y_4 & \geq & 0
\end{array}
$$

Sabemos por el Teorema de Dualidad Fuerte que si el primal tiene SO, también la tiene el dual y sus funciones objetivo coinciden. Además, es fácil ver que el dual tiene como solución factible $(y_1,y_2,y_3,y_4) = (0,0,0,0)$ porque las cantidades de alimentos han de ser no negativas. Por tanto, el primal no puede ser ilimitado. Consecuentemente, el primal debe ser infactible o tener SO, pero si el primal fuera infactible (y por el razonamiento anterior, el dual ilimitado), entonces la restricción sobre la cantidad mínima de fósforo carecería de sentido y se podría simplemente excluir de la formulación (porque no existirían valores posibles para sus coeficientes). Así pues, en cualquier caso, podemos reemplazar el problema primal por su dual en la nueva formulación del problema de la dieta con coeficientes $a_{4j}$ desconocidos.

Por otra parte, si para cualquier solución factible del dual se tiene que $2y_1+y_4 \geq 0.55$, entonces automáticamente esta misma restricción se verificará para su máximo. Por consiguiente, podemos eliminar dicho "máximo" de la restricción sobre la cantidad mínima de fósforo.

Finalmente, el problema que hemos de resolver en este caso queda como sigue:

$$
\begin{array}{rrcl}
    \min & x_{mA} + 0.5x_{A} + 2 x_{mB} + 1.2 x_{S} + 3 x_{L}&\\
    {\rm sujeto\  a}& &\\
    & 78.6x_{mA} + 70.1x_{A}+80.1x_{mB}+67.2x_{S} + 77x_{L} &\geq& 74.2\\
    & 6.50x_{mA} + 9.40x_{A}+8.80x_{mB}+13.7x_{S} + 30.4x_{L} &\geq& 14.7\\
    & 0.02x_{mA} + 0.09x_{A}+0.03x_{mB}+0.14x_{S} + 0.41x_{L} &\geq& 0.14\\
    & 2y_1 + y_4 & \geq & 0.55\\
    & y_1-1.5y_3+y_4 -x_{mA} & \leq      & 0   \\
    &y_1-2y_2  -x_{A}&\leq   & 0    \\
    &y_1-2y_2-1.5y_3 - x_{mB}& \leq   & 0    \\
    &y_1+y_2 - x_{S}& \leq      &  0 \\
    &y_1+y_3 + y_{4} - x_{L} & \leq      & 0  \\
    &x_{mA}, x_{A}, x_{mB}, x_{S}, x_{L}, y_1, y_2, y_3, y_4 & \geq & 0
\end{array}
$$

2.   Resolver dicho problema con *amplpy* y discute la solución obtenida (en comparación con la obtenida en la Práctica 1).



In [19]:
DietaR = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [21]:
# Construimos el problema primal (que podemos coger de la práctica anterior):
DietaR.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set ALIM;                       # Conjunto de alimentos
set NUTR;                       # Conjunto de nutrientes
set Dual_index;                 # Conjunto de índices para las variables duales
param n_min {NUTR} >= 0;        # Cantidad mínima de nutrientes en la dieta
param coste {ALIM} >= 0;        # Coste de cada alimento (por cada 100 g)
param cont {ALIM, NUTR} >= 0;   # Contenido de nutriente en cada alimento
param rest_dual {ALIM, Dual_index}; # Coeficientes de las restricciones del dual
param b_dual {Dual_index};          # Coeficientes FO del dual

var x {ALIM} >= 0;              # Cantidad de cada alimento presente la dieta
                                # (múltiplo de 100 g)
var y {Dual_index} >= 0;        # Variables duales

minimize coste_total:
    sum{a in ALIM} coste[a]*x[a];
s.t. cant_minima {n in NUTR: n != 'Ph'}:
    sum{a in ALIM} cont[a,n]*x[a] >= n_min[n];
    rest_robusta: sum{i in Dual_index} b_dual[i]*y[i] >= n_min['Ph'];
    dual_constraints {a in ALIM}:
    sum{i in Dual_index} rest_dual[a,i]*y[i] - x[a] <= 0;
""")

Introducimos los datos necesarios para resolverel problema robusto "DietaR".

In [22]:
def preparar_datos():
    import pandas as pd
    import numpy as np

    # Creamos un data frame que contenga el coste de los alimentos
    alimentos_df = pd.DataFrame(
        [
            ("MaizA", 1),
            ("Avena", 0.5),
            ("MaizB", 2),
            ("Salvado", 1.2),
            ("Linaza", 3),
        ],
        columns=["ALIMENTO", "coste"],
    ).set_index("ALIMENTO")

    # Creamos un data frame para la cantidad mínima de nutrientes que se
    # requiere en la dieta
    nutr_df = pd.DataFrame(
        [
            ("DN", 74.2),
            ("DP", 14.7),
            ("Ca", 0.14),
            ("Ph", 0.55),
        ],
        columns=["NUTR", "n_min"],
    ).set_index("NUTR")

    # Creamos un data frame para la cantidad de nutrientes que cada
    # alimento contiene
    cont_df = pd.DataFrame(
        np.array(
            [
                [78.6, 70.1, 80.1, 67.2, 77.0],
                [6.50, 9.40, 8.80, 13.7, 30.4],
                [0.02, 0.09, 0.03, 0.14, 0.41],
                [0.27, 0.34, 0.30, 1.29, 0.86],
            ]
        ),
        columns= alimentos_df.index.to_list(),
        index  = nutr_df.index.to_list(),
    )
    cont_df = cont_df.transpose() # OJO, que los ejes (índices) deben coincidir
                                  # con los especificados en la definición del
                                  # parámetro "cont" en el modelo, (ALIM, NUTR)

    # Creamos un data frame para los coeficientes de la FO del dual
    b_dual_df = pd.DataFrame(
        [
            (1, 2),
            (2, 0),
            (3, 0),
            (4, 1),
        ],
        columns=["Dual_index", "b_dual"],
    ).set_index("Dual_index")

    # Creamos un data frame para los coeficientes del problema dual
    rest_dual_df = pd.DataFrame(
        np.array(
            [
                [1, 0, -1.5, 1],
                [1, -2, 0, 0],
                [1, -2, -1.5, 0],
                [1, 1, 0, 0],
                [1, 0, 1, 1],
            ]
        ),
        columns= b_dual_df.index.to_list(),
        index  = alimentos_df.index.to_list(),
    )
    return alimentos_df, nutr_df, cont_df, b_dual_df, rest_dual_df

In [23]:
# Cargamos los datos:

# Load the data from pandas.DataFrame objects:
alimentos_df, nutr_df, cont_df, b_dual_df, rest_dual_df = preparar_datos()

# 1. Enviamos los datos de "alimentos_df" a AMPL e inicializamos el conjunto de índices "ALIM"
DietaR.set_data(alimentos_df, "ALIM")
# 2. Enviamos los datos de "nutr_df" a AMPL e inicializamos el conjunto de índices "NUTR"
DietaR.set_data(nutr_df, "NUTR")
# 3. Fijamos los valores del parámetro "cont" usando "cont_df"
DietaR.get_parameter("cont").set_values(cont_df)
# 4. Enviamos los datos de "b_dual_df" a AMPL e inicializamos el conjunto de índices "Dual_index"
DietaR.set_data(b_dual_df, "Dual_index")
# 5. Fijamos los valores del parámetro "rest_dual" usando "rest_dual_df"
DietaR.get_parameter("rest_dual").set_values(rest_dual_df)

Finalmente resolvemos el problema de la dieta *robustificado*:

In [24]:
DietaR.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert DietaR.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1:  - Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
 - CPXPARAM_Simplex_Display                         0
 - CPXPARAM_MIP_Display                             0
 - CPXPARAM_Barrier_Display                         0
CPLEX 22.1.1: optimal solution; objective 1.621319544
8 simplex iterations


In [26]:
x = DietaR.get_variable("x") # Aquí el valor de "x" es convertido a un data frame de AMPL.
df_x = x.get_values().to_pandas() # Aquí el valor de "x" es convertido a un data frame de Pandas.
y = DietaR.get_variable("y")
df_y = y.get_values().to_pandas()
coste_total = DietaR.get_objective("coste_total").value()
df_x = df_x.round(3)
df_y = df_y.round(3)
print(df_x)
print(df_y)
print(f"Coste total: {coste_total:.3f}")

         x.val
Avena    0.053
Linaza   0.275
MaizA    0.275
MaizB    0.000
Salvado  0.412
   y.val
1  0.275
2  0.138
3  0.000
4  0.000
Coste total: 1.621


3. Calcular el (sobre)coste de la "concatenación de fallos técnicos", esto es, el (sobre)coste de la información incompleta.

El sobrecoste de la información incompleta vendrá dado como la diferencia entre el coste de la solución robusta (1.621) y el coste de la solución con información completa que calculamos en la Práctica 1, esto es, 0.793. Por tanto:

${\rm sobrecoste} = 1.621-0.793 = 0.828$

Es decir, **el coste de la dieta ha crecido más del 104%** como consecuencia de los fallos técnicos en la determinación de la cantidad de fósforo presente en los diferentes alimentos disponibles.